In [1]:
import time
import datetime
import requests
import csv

In [2]:
#!pip install tqdm
from tqdm import notebook

In [3]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [4]:
error_url = []

def FetchDataFromGitHubRestApi(url):
    response = session.get(url)
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [5]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

In [6]:
with open ("not_a_user_or_no_data.txt", "r", encoding="utf8") as f:
    not_a_user_or_no_data = f.readlines()

In [7]:
print(len(all_username), len(not_a_user_or_no_data))

8932 349


In [8]:
for user in not_a_user_or_no_data:
    all_username.remove(user)
print(len(all_username))

8583


In [9]:
all_username = [x.strip() for x in all_username]

In [10]:
user_alias = ["usr"+str(c) for c in range(len(all_username))]

In [11]:
all_event_url = []

for username in all_username:
    for i in (1,2,3):
        all_event_url.append("https://api.github.com/users/"+username+"/events?per_page=100&page="+str(i))

In [12]:
len(all_event_url)

25749

In [16]:
def CollectUserEventData(event_url):
    try:
        event_data = FetchDataFromGitHubRestApi(event_url)
        
        type_ = Select("type", event_data)
        created_at = Select("created_at", event_data)
        return [type_, created_at]
        
    except:
        return ["None", "None"]

In [17]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

now  15 : 1


getting users event data from github rest api need to wait for poll interval (60s) which means you can get the maximum of 3600 requests per hour, therefore there's no need to wait for hours limitation (5000 requests/hr).

In [3]:
25749/60

429.15

since I need to do a total of 25749 requests, I would need to have my computer stays up for ~429.2 hrs.

In [ ]:
user_number = 0
user_event_counter = 3
event_dict = {}

for event_url in notebook.tqdm(all_event_url):
    print("Collect User Event Data from Split", split_number)
    try:
        event_dict[user_alias[user_number]].append(CollectUserEventData(event_url))
    except:
        event_dict[user_alias[user_number]] = []
        event_dict[user_alias[user_number]].append(CollectUserEventData(event_url))
    user_event_counter -= 1
    if user_event_counter == 0:
        user_number += 1
        user_event_counter += 3

print("\ndone")

Collect User Event Data from Split 1


  0%|          | 0/4999 [00:00<?, ?it/s]

In [ ]:
print("write to file")

with open('all_user_event_data.csv', 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "event_type", "created_at"])

    for user in notebook.tqdm(event_dict):
        for user_event in event_dict[user]:
            writer.writerow([user, user_event[0], user_event[1]])

print("done!")

In [ ]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

In [ ]:
4999*60

In [ ]:
30*60

In [ ]:
4999/1000